## Spark Initialize

In [ ]:
# import findspark
# findspark.init()

from pyspark import SparkContext, SparkConf, SQLContext

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import DataFrame as SparkDataFrame
from pyspark.sql import Window



# ml
from pyspark.ml import Pipeline as spk_pipeline
from pyspark.ml.feature import OneHotEncoder as spk_OneHotEncoder, StandardScaler as spk_StandardScaler, VectorAssembler as spk_VectorAssembler
from pyspark.ml.feature import MinMaxScaler as spk_MinMaxScaler, StringIndexer as spk_StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator as spk_RegressionEvaluator

from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param


import os
#https://repo1.maven.org/maven2/com/github/housepower/clickhouse-native-jdbc/2.7.1/clickhouse-native-jdbc-2.7.1.jar
# spark connector https://github.com/ClickHouse/spark-clickhouse-connector
# https://mvnrepository.com/artifact/com.clickhouse
# https://github.com/housepower/ClickHouse-Native-JDBC, For Spark 3.2 and upper, Spark ClickHouse Connector (see upper) is recommended.
packages = [
    "com.clickhouse.spark:clickhouse-spark-runtime-3.5_2.12:0.8.0"
    # "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3"
    ,"com.clickhouse:clickhouse-jdbc:0.7.1-patch1"
    # ,"com.clickhouse:clickhouse-jdbc:0.6.0-patch5"
    ,"com.clickhouse:clickhouse-http-client:0.7.1-patch1"
    # ,"com.clickhouse:clickhouse-http-client:0.6.0-patch5"
    ,"org.apache.httpcomponents.client5:httpclient5:5.3.1"
    # for jdbc 2.7.1 required java 8/11
    # ,"com.github.housepower:clickhouse-native-jdbc:2.7.1"
    # ,"ai.catboost:catboost-spark_3.5_2.12:1.2.7"
    ,"com.microsoft.azure:synapseml_2.12:1.0.8"

]



ram = 16
cpu = 8*3
# Define the application name and setup session
appName = "Connect To ClickHouse via PySpark"
spark = (SparkSession.builder
         .appName(appName)
         # for connetc to external spark master
        #  .master('spark://127.0.0.1:7077')
         .config("spark.jars.packages", ",".join(packages))
        #  .config("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
        #  .config("spark.sql.catalog.clickhouse", "com.clickhouse.spark.ClickHouseCatalog")
        #  .config("spark.sql.catalog.clickhouse.host", CH_IP)
        #  .config("spark.sql.catalog.clickhouse.protocol", "http")
        #  .config("spark.sql.catalog.clickhouse.http_port", "8123")
        #  .config("spark.sql.catalog.clickhouse.user", CH_USER)
        #  .config("spark.sql.catalog.clickhouse.password", CH_PASS)
        #  .config("spark.sql.catalog.clickhouse.database", "default")
        #  .config("spark.spark.clickhouse.write.compression.codec", "lz4")
        #  .config("spark.clickhouse.read.compression.codec", "lz4")
        #  .config("spark.clickhouse.write.format", "arrow")
         #    .config("spark.clickhouse.write.distributed.convertLocal", "true") l
         #    .config("spark.clickhouse.write.repartitionNum", "1") 
         #.config("spark.clickhouse.write.maxRetry", "1000")
         #    .config("spark.clickhouse.write.repartitionStrictly", "true") 
         #    .config("spark.clickhouse.write.distributed.useClusterNodes", "false") 
        #  .config("spark.clickhouse.write.batchSize", "1000000")
         #.config("spark.sql.catalog.clickhouse.socket_timeout", "600000000")
        #  .config("spark.sql.catalog.clickhouse.connection_timeout", "600000000")
        #  .config("spark.sql.catalog.clickhouse.query_timeout", "600000000")
        #  .config("spark.clickhouse.options.socket_timeout", "600000000")
        #  .config("spark.clickhouse.options.connection_timeout", "600000000")
        #  .config("spark.clickhouse.options.query_timeout", "600000000")         
         .config("spark.executor.memory", f"{ram}g")
        #  .config("spark.executor.cores", "5")
         .config("spark.driver.maxResultSize", f"{ram}g")
         .config("spark.driver.memory", f"{ram}g")
         .config("spark.executor.memoryOverhead", f"{ram}g")
        #  .config("spark.sql.debug.maxToStringFields", "100000")
         .getOrCreate()
         )

# LightGBM set config https://microsoft.github.io/SynapseML/docs/Get%20Started/Install%20SynapseML/
# spark.conf.set("spark.jars.repositories", "https://mmlspark.azureedge.net/maven")
# spark.conf.set("spark.jars.excludes", ",".join(exclude_packages))
# spark.conf.set("spark.yarn.user.classpath.first", "true")
# spark.conf.set("spark.sql.parquet.enableVectorizedReader", "false")





# from catboost_spark import CatBoostRegressor as CatBoostRegressor_spark
# from synapse.ml.lightgbm import LightGBMRegressor as LightGBMRegressor_spark


# sp_tools = SparkTools(spark, data_path, tmp_path, CH_USER, CH_PASS, CH_IP)

# spark.sql("use clickhouse")

25/09/03 20:22:54 WARN Utils: Your hostname, cougar resolves to a loopback address: 127.0.1.1; using 192.168.1.150 instead (on interface wlp10s0)
25/09/03 20:22:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/pmu/miniconda3/envs/self-drive/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pmu/.ivy2/cache
The jars for the packages stored in: /home/pmu/.ivy2/jars
com.clickhouse.spark#clickhouse-spark-runtime-3.5_2.12 added as a dependency
com.clickhouse#clickhouse-jdbc added as a dependency
com.clickhouse#clickhouse-http-client added as a dependency
org.apache.httpcomponents.client5#httpclient5 added as a dependency
com.microsoft.azure#synapseml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-626ae638-20df-4b96-b415-a15fc2879d7f;1.0
	confs: [default]
	found com.clickhouse.spark#clickhouse-spark-runtime-3.5_2.12;0.8.0 in central
	found com.clickhouse#clickhouse-jdbc;0.7.1-patch1 in central
	found com.clickhouse#clickhouse-client;0.7.1-patch1 in central
	found com.clickhouse#clickhouse-data;0.7.1-patch1 in central
	found com.clickhouse#clickhouse-http-client;0.7.1-patch1 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.2 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found 

## Описание данных

У компании по доставке еды есть БД в которой содержится таблица заказов пеших курьеров [couriers_orders.parquet](https://drive.google.com/file/d/1ZJlyPkU8W3qlSJI57qYg1e_PQIxCjwZN/view?usp=sharing)  .



In [31]:
couriers_orders = spark.read.parquet("data/couriers_orders.parquet")
couriers_orders.show(5)

+-------------------+----------+--------+--------+-----------+
|               date|courier_id|order_id|distance|travel_time|
+-------------------+----------+--------+--------+-----------+
|2021-07-12 00:00:00|        10|       1|     1.9|      36.17|
|2021-07-02 00:00:00|         3|       2|    3.98|      21.34|
|2021-04-15 00:00:00|         6|       3|    3.98|      43.33|
|2021-07-16 00:00:00|        10|       4|    2.85|      14.01|
|2021-06-11 00:00:00|        10|       5|    4.89|      32.09|
+-------------------+----------+--------+--------+-----------+
only showing top 5 rows



In [32]:
couriers_orders.printSchema()

root
 |-- date: timestamp_ntz (nullable = true)
 |-- courier_id: long (nullable = true)
 |-- order_id: long (nullable = true)
 |-- distance: double (nullable = true)
 |-- travel_time: double (nullable = true)



## EDA

In [33]:
# check null and nan values in each column spark df, nan doesn't work for timestamp dtype
null_nan_counts = []
for c in couriers_orders.columns:
	dtype = dict(couriers_orders.dtypes)[c]
	if dtype in ['double', 'float']:
		expr = F.count(F.when(F.isnull(c) | F.isnan(c), c)).alias(c)
	else:
		expr = F.count(F.when(F.isnull(c), c)).alias(c)
	null_nan_counts.append(expr)
couriers_orders.select(null_nan_counts).show()

+----+----------+--------+--------+-----------+
|date|courier_id|order_id|distance|travel_time|
+----+----------+--------+--------+-----------+
|   0|         0|       0|       0|          0|
+----+----------+--------+--------+-----------+



In [34]:
couriers_orders.select(F.col("distance"), F.col("travel_time")).describe().show()

+-------+------------------+------------------+
|summary|          distance|       travel_time|
+-------+------------------+------------------+
|  count|              1666|              1666|
|   mean|2.7334933973589397| 34.81731692677075|
| stddev|1.2954651948730809|14.475623110887007|
|    min|               0.5|             10.01|
|    max|               5.0|             59.97|
+-------+------------------+------------------+



## Вопрос №1.1:

В конце каждого месяца компания выдает премию для своих курьеров, средняя скорость доставки за прошедший месяц которых больше средней скорости среди всех курьеров. Сколько курьеров получили премию за июнь 2021 года.


In [48]:
# travel time in min and distance in km, add speed column to couriers_orders km/h and filter only june orders
couriers_orders_new = couriers_orders.withColumn("order_speed", F.expr("distance / (travel_time / 60)")) \
    .filter((F.col('date') >= F.lit('2021-06-01')) & (F.col('date') <= F.lit('2021-06-30')))

couriers_orders_new.show(5)

+-------------------+----------+--------+--------+-----------+------------------+
|               date|courier_id|order_id|distance|travel_time|       order_speed|
+-------------------+----------+--------+--------+-----------+------------------+
|2021-06-11 00:00:00|        10|       5|    4.89|      32.09| 9.143035213462136|
|2021-06-14 00:00:00|         4|       9|    4.13|      29.34| 8.445807770961146|
|2021-06-27 00:00:00|         8|      10|    1.04|      12.56| 4.968152866242038|
|2021-06-27 00:00:00|         1|      19|    1.85|      13.56| 8.185840707964601|
|2021-06-28 00:00:00|         2|      25|    4.02|      12.43|19.404666130329844|
+-------------------+----------+--------+--------+-----------+------------------+
only showing top 5 rows



In [75]:
# add column with mean speed by courier_id

# compute mean speed per courier
mean_speed_courier = (
    couriers_orders_new.groupBy("courier_id")
    .agg(F.mean("order_speed").alias("mean_speed_by_courier"))
)

# compute overall mean (as DataFrame, not collected scalar)
overall_mean_df = mean_speed_courier.agg(
    F.mean("mean_speed_by_courier").alias("overall_mean_speed")
)

# Filter couriers whose mean speed is greater than overall mean
couriers_bonus = mean_speed_courier.filter(F.col("mean_speed_by_courier") > overall_mean_df.collect()[0]["overall_mean_speed"])
couriers_bonus.show(), couriers_bonus.count()

+----------+---------------------+
|courier_id|mean_speed_by_courier|
+----------+---------------------+
|         6|    6.791953132084712|
|         1|   6.7493254093092405|
|        10|     6.58047952209051|
|         8|    6.826371958342846|
|         2|    6.528423151187261|
|         4|    7.692341132350769|
+----------+---------------------+



(None, 6)